## JG Thoughts 
- Unemployment rate vs employment #s? Look at change in UE rate or total as of certain point in time
- Inc_Per_CAp_2022 -> typo?
- Population Density per mi² -> what yr? Also is string
- NA values in NJ
- Convert weeks into %?

- Maybe PCA for the white, black, Hispanic stats? But missing a decent chunk it looks like. Maybe only black or white. But a lot of 0s so maybe just exclde entirely  
- Maybe do something to 'Population' like catgy 1-5 for large small or medium. But can leave for now I guess 

- Do SVM or NN models? SVM mostly for classification. NN maybe not enough data

## Data Cleaning & Preprocessing

In [101]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import VotingRegressor

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor


In [102]:
df = pd.read_csv('Data/Cleaned/county_df.csv')
df.head()

,FIPS_x,County,Years of Potential Life Lost Rate (premature death),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair/Poor Health,percent_smokers,percent_obese,Food Environment Index,...,cases_2020,cases_2021,cases_2022,deaths_2020,deaths_2021,deaths_2022,Masks,FIPS_y,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_65PlusPop_Pct
0,1001,Autauga,8824.0,10471.0,NaN,8707.0,18,19,38,7.2,...,4190.0,11018.0,18961.0,5631.0,47405.0,124934.0,267.0,1001,42.2,73.8
1,1003,Baldwin,7225.0,10042.0,3087.0,7278.0,18,17,31,8.0,...,13601.0,39911.0,67496.0,12412.0,148723.0,397246.0,267.0,1003,53.2,89.9
2,1005,Barbour,9586.0,11333.0,NaN,7310.0,26,22,44,5.6,...,1514.0,3860.0,7027.0,2035.0,24364.0,60044.0,267.0,1005,44.5,75.3
3,1007,Bibb,11784.0,14813.0,NaN,11328.0,20,20,38,7.6,...,1834.0,4533.0,7692.0,2678.0,28085.0,65964.0,267.0,1007,36.6,64.2
4,1009,Blount,10908.0,NaN,5620.0,11336.0,21,20,34,8.5,...,4641.0,11256.0,17731.0,3855.0,56300.0,144559.0,267.0,1009,31.9,56.6


In [103]:
# df[['cases_2020', 'cases_2021', 'cases_2022', 'deaths_2020', 'deaths_2021', 'deaths_2022', 'Population']]

In [104]:
# Drop rows that we will not be using
df.drop(columns = ['County', 'YPLL Rate (Black)', 'YPLL Rate (Hispanic)', 'YPLL Rate (White)', 'Number Uninsured', 'Number Primary Care Physicians', 'FIPS_y',
                        'Preventable Hosp. Rate (Black)', 'Preventable Hosp. Rate (Hispanic)', 'Preventable Hosp. Rate (White)',  'Percent Vaccinated Flu (Black)', 
                        'Percent  Vaccinated (Hispanic) Flu', 'Percent Vaccinated (White) Flu', 'Number Some College', 'Number Unemployed', 'Labor Force', 'PCP Ratio',
                        '80th Percentile Income', '20th Percentile Income', '95% CI - Low', '95% CI - High', 'Life Expectancy (Black)', 'Life Expectancy (Hispanic)', 
                        'Life Expectancy (White)', 'Number HIV Cases', 'Household income (Black)', 'Household income (Hispanic)', 'Household income (White)'], inplace = True)

In [105]:
# df.columns
# df.dtypes
df.shape

(3001, 55)

In [106]:
# Make FIPS index 
df.set_index('FIPS_x', inplace=True)

In [107]:
# # Create new columns for per populaltion stats - YPL, Number pre-mature Deaths, Number rural 
# df['YPL'] = df['Years of Potential Life Lost Rate (premature death)']/df['Population']
# df['pre mature deaths'] = df['Number pre-mature Deaths']/df['Population']
# df['rural'] = df['number Rural']/df['Population']

# df.drop(columns = ['Years of Potential Life Lost Rate (premature death)', 'Number pre-mature Deaths', 'number Rural'], inplace = True)

In [108]:
# Dummify State and Presence of water violation
# df = pd.get_dummies(columns = ['State'], data = df, drop_first=True)
df['water'] = df['Presence of water violation'].map({'No': 0, 'Yes': 1})
df.drop(columns = ['Presence of water violation', 'State'], inplace = True)

In [109]:
# Calculate total cases and deaths, and convert to % of population 
df['cases'] = df['cases_2022']
df['deaths'] = df['deaths_2022']

# df['case_rate'] = df['cases']/df['Population']
# df['death_rate'] = df['deaths']/df['Population']

# Deaths seem to be off? More deaths than population 
df.drop(columns = ['cases_2020', 'cases_2021', 'cases_2022', 'deaths_2020', 'deaths_2021', 'deaths_2022'], inplace = True)

In [110]:
# Drop Na values (3k rows -> 1828)
df.dropna(inplace=True)
df.shape

(1828, 49)

## Linear Regression (LR)

Add in SW reg? https://www.geeksforgeeks.org/stepwise-regression-in-python/

In [111]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [112]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [113]:
print(f'Train Score: {lr.score(X_train, y_train)}')
print(f'Test Score: {lr.score(X_test, y_test)}')

Train Score: 0.968148387444034
Test Score: 0.9325206419357982


In [114]:
pd.DataFrame(set(zip(X.columns, lr.coef_)), columns = ['Variable Name', 'Coefficient']).sort_values('Coefficient').round(1)

,Variable Name,Coefficient
34,percent Native Hawaiian/Other Pacific Islander,-3166.5
3,percent Asian,-2008.7
38,Percent Uninsured Adults,-1573.8
31,Percent Unemployed,-1266.1
27,Percent Uninsured children,-851.9
8,% Fair/Poor Health,-677.5
46,Average Daily PM2.5,-640.1
1,Income Ratio,-494.9
9,percent_smokers,-275.7
17,percent_obese,-199.6


## K Nearest Neighbors (KNN)

In [115]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [116]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [117]:
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train_sc, y_train)

KNeighborsRegressor(n_neighbors=10)

In [118]:
print(f'Train Score: {knn.score(X_train_sc, y_train)}')
print(f'Test Score: {knn.score(X_test_sc, y_test)}')

Train Score: 0.6724283832895974
Test Score: 0.6910618916621735


## Decision Trees (DT)

In [119]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [120]:
%%time
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

CPU times: total: 750 ms
Wall time: 90.5 ms


DecisionTreeRegressor()

In [121]:
print(f'Train Score: {tree.score(X_train, y_train)}')
print(f'Test Score: {tree.score(X_test, y_test)}')

Train Score: 1.0
Test Score: 0.8743083940706999


In [122]:
# Top 10 features
pd.DataFrame({'feature': X.columns, 'importance': tree.feature_importances_}).sort_values('importance', ascending = False).head(10)

,feature,importance
31,Population,0.985263
17,Severe Housing Cost Burden,0.003179
38,percent Hispanic,0.002835
9,Preventable Hosp stays Rate,0.001315
34,percent African American,0.001297
44,Administered_Dose1_Pop_Pct,0.000876
27,Percent Uninsured children,0.000581
23,HIV Prevalence Rate,0.000380
2,percent_smokers,0.000347
12,Percent Some College,0.000343


### Bagged Decision Tree

In [123]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [124]:
pgrid = {
    'estimator__max_depth': [20, 30, 40, 50],
    # 'max_samples' : [0.5, 0.7, 0.9],
}

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
bag = BaggingRegressor(tree, n_estimators = 500, random_state = 42)
tree_bagged = GridSearchCV(bag, pgrid, cv = 5, n_jobs = 8)
# tree_bagged.get_params()

In [125]:
%%time
tree_bagged.fit(X_train, y_train)

CPU times: total: 22.1 s
Wall time: 1min 41s


GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                        n_estimators=500, random_state=42),
             n_jobs=8, param_grid={'estimator__max_depth': [20, 30, 40, 50]})

In [126]:
print(f'Train Score: {tree_bagged.score(X_train, y_train)}')
print(f'Test Score: {tree_bagged.score(X_test, y_test)}')

Train Score: 0.945974063923793
Test Score: 0.8251022410910961


In [127]:
tree_bagged.best_params_

{'estimator__max_depth': 30}

## Random Forest (RF)

In [128]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [129]:
rf = RandomForestRegressor(
    n_estimators=500,
    random_state = 42,
)

In [130]:
%%time
rf.fit(X_train, y_train)

CPU times: total: 21.6 s
Wall time: 22.2 s


RandomForestRegressor(n_estimators=500, random_state=42)

In [131]:
print(f'Train Score: {rf.score(X_train, y_train)}')
print(f'Test Score: {rf.score(X_test, y_test)}')

Train Score: 0.9466441924694832
Test Score: 0.823931592532311


In [132]:
pd.DataFrame({'Features': X.columns, 'Importance': rf.feature_importances_}).sort_values('Importance', ascending = False)

,Features,Importance
31,Population,0.754313
21,Number pre-mature Deaths,0.192842
16,Percent Severe Housing Problems,0.016876
19,Inadequate Facilities,0.009961
39,percent Non-Hispanic White,0.003844
18,Overcrowding,0.003163
38,percent Hispanic,0.001648
2,percent_smokers,0.000940
6,percent Excessive Drinking,0.000934
29,Segregation index black/white,0.000890


### RF w/ RandomSearchCV

In [133]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [134]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': [None, 10, 20, 30, 40, 50]
}

rf = RandomForestRegressor(
    n_estimators=500,
    random_state = 42,
)


rf_rs = RandomizedSearchCV(rf, params, n_iter=100, cv = 5, n_jobs = 8)
# rf_rs.get_params()

In [135]:
%%time
rf_rs.fit(X_train, y_train)

CPU times: total: 8.3 s
Wall time: 11min 42s


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_estimators=500,
                                                   random_state=42),
                   n_iter=100, n_jobs=8,
                   param_distributions={'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47]),
                                        'min_samples_leaf': [1, 2, 3, 4, 5]})

In [136]:
print(f'Train Score: {rf_rs.score(X_train, y_train)}')
print(f'Test Score: {rf_rs.score(X_test, y_test)}')

Train Score: 0.8487903126953178
Test Score: 0.8963689141717528


In [137]:
rf_rs.best_params_

{'min_samples_leaf': 2, 'max_features': 20, 'max_depth': 30}

### RF w/ Adaboost

In [138]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [139]:
rf_ada = AdaBoostRegressor(
    RandomForestRegressor(),
    n_estimators = 500
)    

In [140]:
%%time
rf_ada.fit(X_train, y_train)

CPU times: total: 13min 55s
Wall time: 14min 13s


AdaBoostRegressor(estimator=RandomForestRegressor(), n_estimators=500)

In [141]:
print(f'Train Score: {rf_ada.score(X_train, y_train)}')
print(f'Test Score: {rf_ada.score(X_test, y_test)}')

Train Score: 0.9996945687358259
Test Score: 0.8436961233119971


#### RF w/ Adaboost & RandomSearchCV

In [142]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [143]:
params = {
    'estimator__max_features': np.arange(5, X.shape[1] + 1),
    'estimator__min_samples_leaf': [1, 2, 3],
    'estimator__max_depth': [None, 10, 20, 30, 40, 50]
}

rf_ada = AdaBoostRegressor(
    RandomForestRegressor(),
    n_estimators = 500
)    


rf_ada_rs = RandomizedSearchCV(rf_ada, params, n_iter=20, cv = 5, n_jobs = 8)
# rf_ada_rs.get_params()

In [144]:
# rf_ada_rs.fit(X_train, y_train)

In [145]:
# print(f'Train Score: {rf_ada_rs.score(X_train, y_train)}')
# print(f'Test Score: {rf_ada_rs.score(X_test, y_test)}')

In [146]:
# rf_ada_rs.best_params_

### RF w/ Gradboost

In [147]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [148]:
rf_gb = GradientBoostingRegressor()

In [149]:
%%time
rf_gb.fit(X_train, y_train)

CPU times: total: 1.05 s
Wall time: 1.09 s


GradientBoostingRegressor()

In [150]:
print(f'Train Score: {rf_gb.score(X_train, y_train)}')
print(f'Test Score: {rf_gb.score(X_test, y_test)}')

Train Score: 0.9981189104539843
Test Score: 0.8845506577705898


#### RF w/ Gradboost & RandomSearchCV

In [151]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [152]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    # 'estimator__min_samples_leaf': [1, 2, 3],
    'max_depth': [None, 10, 20, 30, 40, 50]
}

rf_gb = GradientBoostingRegressor()


rf_gb_rs = RandomizedSearchCV(rf_gb, params, n_iter=100, cv = 5, n_jobs = 8)
# rf_gb_rs.get_params()

In [153]:
%%time
rf_gb_rs.fit(X_train, y_train)

CPU times: total: 1.89 s
Wall time: 4min 56s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=100,
                   n_jobs=8,
                   param_distributions={'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47])})

In [154]:
print(f'Train Score: {rf_gb_rs.score(X_train, y_train)}')
print(f'Test Score: {rf_gb_rs.score(X_test, y_test)}')

Train Score: 0.9999997634134768
Test Score: 0.7766914379956544


In [155]:
rf_gb_rs.best_params_

{'max_features': 18, 'max_depth': 10}

## Extra Trees (ET)

In [156]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [157]:
%%time
et = ExtraTreesRegressor(n_estimators = 500)
et.fit(X_train, y_train)

CPU times: total: 11 s
Wall time: 11.3 s


ExtraTreesRegressor(n_estimators=500)

In [158]:
print(f'Train Score: {et.score(X_train, y_train)}')
print(f'Test Score: {et.score(X_test, y_test)}')

Train Score: 1.0
Test Score: 0.8469666185960503


#### ET w/ RandomSearchCV

In [159]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [160]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3],
    'max_depth': [None, 10, 20, 30, 40, 50]
}

et = ExtraTreesRegressor(n_estimators = 500)

et_rs = RandomizedSearchCV(et, params, n_iter=100, cv = 5, n_jobs = 8)
# et_rs.get_params()

In [161]:
%%time
et_rs.fit(X_train, y_train)

CPU times: total: 2.11 s
Wall time: 4min 24s


RandomizedSearchCV(cv=5, estimator=ExtraTreesRegressor(n_estimators=500),
                   n_iter=100, n_jobs=8,
                   param_distributions={'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47]),
                                        'min_samples_leaf': [1, 2, 3]})

In [162]:
print(f'Train Score: {et_rs.score(X_train, y_train)}')
print(f'Test Score: {et_rs.score(X_test, y_test)}')

Train Score: 0.8965464646904658
Test Score: 0.9183893600656411


In [163]:
et_rs.best_params_

{'min_samples_leaf': 2, 'max_features': 35, 'max_depth': 10}

## Ensembled Model (Voting Regressor)

In [164]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [165]:
vr = VotingRegressor([
    ('lr', lr),
    ('tree', tree_bagged),
    ('rf_rs', rf_rs),
    ('rf_gb', rf_gb),
    ('et_rs', et_rs)
])

In [166]:
%%time
vr.fit(X_train, y_train)

CPU times: total: 40.7 s
Wall time: 18min 58s


VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('tree',
                             GridSearchCV(cv=5,
                                          estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                                                     n_estimators=500,
                                                                     random_state=42),
                                          n_jobs=8,
                                          param_grid={'estimator__max_depth': [20,
                                                                               30,
                                                                               40,
                                                                               50]})),
                            ('rf_rs',
                             RandomizedSearchCV(cv=5,
                                                estimator=RandomForestRegressor(n_estimators=500,
                                                                                random_state=42),
                                                n_iter=100, n_jo...
                            ('rf_gb', GradientBoostingRegressor()),
                            ('et_rs',
                             RandomizedSearchCV(cv=5,
                                                estimator=ExtraTreesRegressor(n_estimators=500),
                                                n_iter=100, n_jobs=8,
                                                param_distributions={'max_depth': [None,
                                                                                   10,
                                                                                   20,
                                                                                   30,
                                                                                   40,
                                                                                   50],
                                                                     'max_features': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47]),
                                                                     'min_samples_leaf': [1,
                                                                                          2,
                                                                                          3]}))])

In [167]:
print(f'Train Score: {vr.score(X_train, y_train)}')
print(f'Train Score: {vr.score(X_test, y_test)}')

Train Score: 0.9567849520235566
Train Score: 0.9067405328360387
